In [24]:
import hopsworks
import time
from datetime import datetime
from datetime import timedelta

fs = hopsworks.login().get_feature_store()

Connection closed.
2025-10-09 20:45:56,929 INFO: Python Engine initialized.

Logged in to project, explore it here https://snurran.devnet.hops.works/p/4217


### Get some sample user_ids for online inference tests
Read them up from `user_clicks`

In [25]:
num_samples=100

clicks_fg = fs.get_feature_group("user_clicks")
df = clicks_fg.read()


max_event_time = df['event_time'].max() 
min_event_time = df['event_time'].min() 
max_event_time_dt = datetime.fromtimestamp(max_event_time)

one_min = int((max_event_time_dt - timedelta(minutes=1)).timestamp())
ten_min = int((max_event_time_dt - timedelta(minutes=10)).timestamp())
thirty_min = int((max_event_time_dt - timedelta(minutes=30)).timestamp())
one_hr = int((max_event_time_dt - timedelta(hours=1)).timestamp())

sampled_user_ids = df.sample(n=num_samples, random_state=42)

2025-10-09 20:45:58,431 WARNING: VersionWarning: No version provided for getting feature group `user_clicks`, defaulting to `1`.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.23s) 


In [26]:
import pymysql

host = "onlinefs.mysql.service.consul"
port = 3306
database = "aggregations"
user = "aggregations_jim00000"
password = "tRwvOWqSFNUOaIXzXyaIzDIfIKxjQSzG"

conn = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database,
    ssl=None 
)


start = time.perf_counter()
total_count = 0
for idx, row in sampled_user_ids.iterrows():
    user_id = row['user_id']
    event_time = row['event_time']
    cursor = conn.cursor()
    query = "SELECT * FROM windowed_click_counts_1 WHERE user_id = %s AND event_time > %s"
    cursor.execute(query, (user_id, one_hr))
    rows = cursor.fetchall()
    cursor.close()
    click_count_1_min = 0
    click_count_10_min = 0
    click_count_30_min = 0
    click_count_1_hour = 0

    # Compute windowed aggregation (count)
    for r in rows:
        evt_time = r[1]
        delta = evt_time - event_time
        if delta <= 60: 
            click_count_1_min += 1
        if delta <= 600:
            click_count_10_min += 1
        if delta <= 1800:
            click_count_30_min += 1
        if delta <= 3600:
            click_count_1_hour += 1    
    print(
        f"user_id={user_id}, "
        f"click_count_1_min={click_count_1_min}, "
        f"click_count_10_min={click_count_10_min}, "
        f"click_count_30_min={click_count_30_min}, "
        f"click_count_1_hour={click_count_1_hour}"
    )
end = time.perf_counter()
print(f"Time taken (s): {(end-start)}")

conn.close()


user_id=44, click_count_1_min=51, click_count_10_min=201, click_count_30_min=500, click_count_1_hour=922
user_id=8, click_count_1_min=196, click_count_10_min=334, click_count_30_min=655, click_count_1_hour=870
user_id=47, click_count_1_min=43, click_count_10_min=165, click_count_30_min=438, click_count_1_hour=860
user_id=82, click_count_1_min=718, click_count_10_min=859, click_count_30_min=876, click_count_1_hour=876
user_id=76, click_count_1_min=490, click_count_10_min=630, click_count_30_min=842, click_count_1_hour=842
user_id=90, click_count_1_min=188, click_count_10_min=321, click_count_30_min=594, click_count_1_hour=873
user_id=4, click_count_1_min=750, click_count_10_min=875, click_count_30_min=888, click_count_1_hour=888
user_id=49, click_count_1_min=342, click_count_10_min=470, click_count_30_min=736, click_count_1_hour=846
user_id=11, click_count_1_min=824, click_count_10_min=859, click_count_30_min=859, click_count_1_hour=859
user_id=60, click_count_1_min=653, click_count_10_